In [5]:
import sqlite3
from flask import Flask, request, jsonify
import threading
import random
import string
import json
import logging
from flask_cors import CORS

# Logging konfigurieren
logging.basicConfig(filename='api.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ========================================
# 1. DATENBANK SETUP
# ========================================

def init_database():
    """Erstellt die Datenbank mit der definierten Struktur"""
    conn = sqlite3.connect('boxes.db')
    cursor = conn.cursor()
   
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS boxes (
            CODE TEXT PRIMARY KEY NOT NULL,
            Location TEXT,
            Content TEXT
        )
    ''')
   
    conn.commit()
    conn.close()
    print("✓ Datenbank erstellt")

def generate_unique_code():
    """Generiert einen eindeutigen, zufälligen Code aus Buchstaben und Zahlen"""
    code = ''.join(random.choices(string.ascii_uppercase, k=2)) + ''.join(random.choices(string.digits, k=2))
    return code

# ========================================
# 2. FLASK REST API
# ========================================

app = Flask(__name__)
CORS(app)  # CORS aktivieren für Cross-Origin-Requests

def get_db():
    """Hilfsfunktion für Datenbankverbindung"""
    conn = sqlite3.connect('boxes.db')
    conn.row_factory = sqlite3.Row
    return conn

In [2]:
import requests

# Basis-URL der API
BASE_URL = 'http://localhost:5006/api'

# ========================================
# TEST-CODE FÜR DIE API
# ========================================

def test_create_box():
    """Test: Kiste erstellen"""
    data = {'Location': 'Test-Lager', 'Content': {'item': 'Test-Hammer', 'quantity': 1}}
    response = requests.post(f'{BASE_URL}/boxes', json=data)
    print(f'Erstellen: {response.status_code} - {response.json()}')
    return response.json().get('CODE') if response.status_code == 201 else None

def test_get_all_boxes():
    """Test: Alle Kisten abrufen"""
    response = requests.get(f'{BASE_URL}/boxes')
    print(f'Alle abrufen: {response.status_code} - {len(response.json())} Kisten gefunden')

def test_get_box(code):
    """Test: Einzelne Kiste abrufen"""
    response = requests.get(f'{BASE_URL}/boxes/{code}')
    print(f'Kiste {code} abrufen: {response.status_code} - {response.json()}')

def test_update_box(code):
    """Test: Kiste aktualisieren"""
    data = {'Content': {'item': 'Test-Schraube', 'quantity': 5}}
    response = requests.put(f'{BASE_URL}/boxes/{code}', json=data)
    print(f'Aktualisieren {code}: {response.status_code} - {response.json()}')

def test_delete_box(code):
    """Test: Kiste löschen"""
    response = requests.delete(f'{BASE_URL}/boxes/{code}')
    print(f'Löschen {code}: {response.status_code} - {response.json()}')

def test_stats():
    """Test: Statistiken abrufen"""
    response = requests.get(f'{BASE_URL}/stats')
    print(f'Statistiken: {response.status_code} - {response.json()}')

def test_filter_boxes():
    """Test: Gefilterte Suche"""
    response = requests.get(f'{BASE_URL}/boxes?location=Test-Lager')
    print(f'Filtern nach Location: {response.status_code} - {len(response.json())} Kisten gefunden')

def test_validation():
    """Test: Validierung (leere Location)"""
    data = {'Content': {'item': 'Test'}}
    response = requests.post(f'{BASE_URL}/boxes', json=data)
    print(f'Validierung (leere Location): {response.status_code} - {response.json()}')

# ========================================
# TESTS AUSFÜHREN
# ========================================

if __name__ == '__main__':
    print("=== API-Tests starten ===")
    
    # 1. Validierung testen
    test_validation()
    
    # 2. Kiste erstellen
    code = test_create_box()
    if code:
        # 3. Einzelne Kiste abrufen
        test_get_box(code)
        
        # 4. Aktualisieren
        test_update_box(code)
        
        # 5. Gefilterte Suche
        test_filter_boxes()
        
        # 6. Alle abrufen
        test_get_all_boxes()
        
        # 7. Statistiken
        test_stats()
        
        # 8. Löschen
        test_delete_box(code)
    
    print("=== Tests beendet ===")

=== API-Tests starten ===
Validierung (leere Location): 400 - {'error': 'Location darf nicht leer sein'}
Erstellen: 201 - {'CODE': 'ZH42', 'message': 'Kiste erstellt'}
Kiste ZH42 abrufen: 200 - {'CODE': 'ZH42', 'Content': {'item': 'Test-Hammer', 'quantity': 1}, 'Location': 'Test-Lager'}
Aktualisieren ZH42: 200 - {'message': 'Kiste aktualisiert'}
Filtern nach Location: 200 - 1 Kisten gefunden
Alle abrufen: 200 - 4 Kisten gefunden
Statistiken: 200 - {'locations': {'Keller': 1, 'Lager': 2, 'Test-Lager': 1}, 'total_boxes': 4}
Löschen ZH42: 200 - {'message': 'Kiste gelöscht'}
=== Tests beendet ===


Hier sind alle wichtigen Befehle (hauptsächlich cURL für API-Tests), um dein Projekt zu testen. Dein Server läuft auf http://localhost:5006. Ich habe die Befehle nach Endpunkten geordnet und mit Beispielen versehen. Kopiere sie einfach in dein Terminal und führe sie aus.

### 1. **Server starten (falls nicht läuft)**
   - Im Notebook die Zelle ausführen (bereits integriert).
   - Oder manuell: `python -c "from kiste import *; start_server()"` (aber besser das Notebook verwenden).

### 2. **Kiste erstellen (POST /api/boxes)**
   - **Erfolgreich mit Location und Content:**
     ```
     curl -X POST http://localhost:5006/api/boxes -H "Content-Type: application/json" -d '{"Location": "Lager", "Content": {"item": "Hammer", "quantity": 5}}'
     ```
     → Gibt `{"message": "Kiste erstellt", "CODE": "AB12"}` zurück.

   - **Mit eigenem CODE:**
     ```
     curl -X POST http://localhost:5006/api/boxes -H "Content-Type: application/json" -d '{"CODE": "XY99", "Location": "Büro", "Content": {"item": "Papier"}}'
     ```

   - **Fehler: Ohne Location (Validierung testen):**
     ```
     curl -X POST http://localhost:5006/api/boxes -H "Content-Type: application/json" -d '{"Content": {"item": "Test"}}'
     ```
     → Gibt `{"error": "Location darf nicht leer sein"}` zurück.

   - **Fehler: Ungültiges JSON in Content:**
     ```
     curl -X POST http://localhost:5006/api/boxes -H "Content-Type: application/json" -d '{"Location": "Lager", "Content": "kein json"}'
     ```
     → Gibt Fehler zurück.

### 3. **Alle Kisten abrufen (GET /api/boxes)**
   - **Alle Kisten:**
     ```
     curl http://localhost:5006/api/boxes
     ```
     → Gibt Array mit allen Kisten zurück, z. B. `[{"CODE": "AB12", "Location": "Lager", "Content": {"item": "Hammer", "quantity": 5}}]`.

   - **Filtern nach Location (Suchfunktion):**
     ```
     curl "http://localhost:5006/api/boxes?location=Lager"
     ```
     → Zeigt nur Kisten mit "Lager" in Location.

   - **Filtern nach Content-Item (Suchfunktion):**
     ```
     curl "http://localhost:5006/api/boxes?content_item=Hammer"
     ```
     → Zeigt Kisten, deren Content "Hammer" enthält.

   - **Kombinierte Filter:**
     ```
     curl "http://localhost:5006/api/boxes?location=Lager&content_item=Hammer"
     ```

### 4. **Einzelne Kiste abrufen (GET /api/boxes/<code>)**
   - **Erfolgreich:**
     ```
     curl http://localhost:5006/api/boxes/AB12
     ```
     → Gibt die Kiste zurück.

   - **Fehler: Nicht gefunden:**
     ```
     curl http://localhost:5006/api/boxes/ZZ99
     ```
     → Gibt `{"error": "Kiste nicht gefunden"}` zurück.

### 5. **Kiste aktualisieren (PUT /api/boxes/<code>)**
   - **Location und Content aktualisieren:**
     ```
     curl -X PUT http://localhost:5006/api/boxes/AB12 -H "Content-Type: application/json" -d '{"Location": "Neuer Ort", "Content": {"item": "Schraube", "quantity": 10}}'
     ```
     → Gibt `{"message": "Kiste aktualisiert"}` zurück.

   - **Nur Location aktualisieren:**
     ```
     curl -X PUT http://localhost:5006/api/boxes/AB12 -H "Content-Type: application/json" -d '{"Location": "Büro"}'
     ```

   - **Nur Content aktualisieren:**
     ```
     curl -X PUT http://localhost:5006/api/boxes/AB12 -H "Content-Type: application/json" -d '{"Content": {"item": "Nagel"}}'
     ```

   - **Fehler: Leere Location:**
     ```
     curl -X PUT http://localhost:5006/api/boxes/AB12 -H "Content-Type: application/json" -d '{"Location": ""}'
     ```
     → Gibt `{"error": "Location darf nicht leer sein"}` zurück.

   - **Fehler: Keine Daten:**
     ```
     curl -X PUT http://localhost:5006/api/boxes/AB12 -H "Content-Type: application/json" -d '{}'
     ```
     → Gibt `{"error": "Keine Daten zum Aktualisieren angegeben"}` zurück.

### 6. **Kiste löschen (DELETE /api/boxes/<code>)**
   - **Erfolgreich:**
     ```
     curl -X DELETE http://localhost:5006/api/boxes/AB12
     ```
     → Gibt `{"message": "Kiste gelöscht"}` zurück.

   - **Fehler: Nicht gefunden:**
     ```
     curl -X DELETE http://localhost:5006/api/boxes/ZZ99
     ```
     → Gibt `{"error": "Kiste nicht gefunden"}` zurück.

### 7. **Statistiken abrufen (GET /api/stats)**
   - ```
     curl http://localhost:5006/api/stats
     ```
     → Gibt z. B. `{"total_boxes": 3, "locations": {"Lager": 2, "Büro": 1}}` zurück.

### 8. **Logs anzeigen (Logging testen)**
   - **Logs im Terminal anzeigen:**
     ```
     cd "/Users/momu/Python/datei kalender" && cat api.log
     ```
     → Zeigt alle Log-Einträge (z. B. "Kiste AB12 erstellt", Warnungen, Fehler).

   - **Letzte 10 Zeilen:**
     ```
     cd "/Users/momu/Python/datei kalender" && tail -10 api.log
     ```

### Tipps zum Testen:
- Ersetze `<code>` mit echten CODES aus deinen Tests (z. B. "AB12").
- Verwende `-v` bei cURL für detaillierte Ausgaben: `curl -v ...`
- Nach Tests kannst du die Datenbank zurücksetzen, indem du `boxes.db` löschst und das Notebook neu startest.
- Wenn der Server nicht läuft, führe die Notebook-Zelle erneut aus.

Falls du spezifische Befehle für einen Endpunkt brauchst oder Hilfe bei Fehlern hast, lass es mich wissen!### Tipps zum Testen:
- Ersetze `<code>` mit echten CODES aus deinen Tests (z. B. "AB12").
- Verwende `-v` bei cURL für detaillierte Ausgaben: `curl -v ...`
- Nach Tests kannst du die Datenbank zurücksetzen, indem du `boxes.db` löschst und das Notebook neu startest.
- Wenn der Server nicht läuft, führe die Notebook-Zelle erneut aus.

Falls du spezifische Befehle für einen Endpunkt brauchst oder Hilfe bei Fehlern hast, lass es mich wissen!

In [7]:
import sqlite3
from flask import Flask, request, jsonify
import threading
import random
import string
import json
import logging
from flask_cors import CORS

logging.basicConfig(filename='api.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def init_database():
    conn = sqlite3.connect('boxes.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS boxes (
            CODE TEXT PRIMARY KEY NOT NULL,
            Location TEXT,
            Content TEXT
        )
    ''')
    conn.commit()
    conn.close()
    print('✓ Datenbank erstellt')

def generate_unique_code():
    code = ''.join(random.choices(string.ascii_uppercase, k=2)) + ''.join(random.choices(string.digits, k=2))
    return code

app = Flask(__name__)
CORS(app)

def get_db():
    conn = sqlite3.connect('boxes.db')
    conn.row_factory = sqlite3.Row
    return conn

# Hier die Routen einfügen, aber kurz für Test
@app.route('/api/boxes', methods=['GET'])
def get_all_boxes():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM boxes')
    boxes = [dict(row) for row in cursor.fetchall()]
    for box in boxes:
        try:
            box['Content'] = json.loads(box['Content'])
        except:
            box['Content'] = {}
    conn.close()
    return jsonify(boxes), 200

init_database()

def start_server():
    app.run(host='0.0.0.0', port=5006, debug=False, use_reloader=False)

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

print('✓ Server läuft auf http://0.0.0.0:5006')
print('  Zugriff aus Netzwerk: http://<deine-ip>:5006')

✓ Datenbank erstellt
✓ Server läuft auf http://0.0.0.0:5006
  Zugriff aus Netzwerk: http://<deine-ip>:5006


 * Serving Flask app '__main__'
 * Debug mode: off
